### Automating Trading Strategies

In [5]:
import pandas as pd
from binance.client import Client 
from binance import ThreadedWebsocketManager

In [7]:

## https://testnet.binance.vision/   to get api resets every month
key_test= 'FupfPBWYCoWMRfPVOlx7pCkz7DuTh5gznBSE5rwO5pa7FCMf2IFKsZS2SXMMj6Zr'

secret_test = 'vBbzzVbHcuuHBrpi1yjKYFWqSEHk5iXbgwk52FCiWJVmqRASusGQNSSBDNAlXyaK'

In [8]:
client = Client(key_test, secret_test,tld='us',testnet=True) #initialize client 

In [9]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1645761041984,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.90000000', 'locked': '0.20000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '3079.82159717', 'locked': '3000.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [15]:
twm = ThreadedWebsocketManager()
twm.start()

In [16]:
df = pd.DataFrame(columns=['Open','High','Low','Close','Volume','Complete'])

In [17]:
def stream_candles(msg):
    
    event_time = pd.to_datetime(msg['E'],unit='ms')
    start_time = pd.to_datetime(msg['k']['t'],unit='ms')
    first = float(msg['k']['o'])
    high = float(msg['k']['h'])
    low = float(msg['k']['l'])
    close = float(msg['k']['c'])
    volume = float(msg['k']['v'])
    complete = float(msg['k']['x'])
    
    print("Time: {} | Price: {}".format(event_time, close))
    
    df.loc[start_time] = [first, high, low, close, volume, complete]

In [18]:
twm.start_kline_socket(callback=stream_candles,symbol='BTCUSDT',interval ='1m')

'btcusdt@kline_1m'

Time: 2022-02-25 17:41:13.628000 | Price: 39130.01
Time: 2022-02-25 17:41:15.756000 | Price: 39124.89
Time: 2022-02-25 17:41:18.332000 | Price: 39117.63
Time: 2022-02-25 17:41:20.413000 | Price: 39117.16
Time: 2022-02-25 17:41:22.670000 | Price: 39121.52
Time: 2022-02-25 17:41:25.091000 | Price: 39114.19
Time: 2022-02-25 17:41:27.127000 | Price: 39117.63
Time: 2022-02-25 17:41:29.356000 | Price: 39121.52
Time: 2022-02-25 17:41:31.370000 | Price: 39127.66


In [19]:
twm.stop()

In [20]:
df

,Open,High,Low,Close,Volume,Complete
2022-02-25 17:41:00,39130.01,39137.00,39108.40,39124.43,12.17729,1.0
2022-02-25 17:42:00,39124.43,39124.44,39124.43,39124.43,0.28216,0.0


Time: 2022-02-25 17:42:04.270000 | Price: 39124.44


In [36]:
#building the LongOnlyTrader Class 
class LongOnlyTrader():
    
    def __init__(self,symbol,bar_length):
        self.symbol = symbol
        self.bar_length = bar_length
        self.data = pd.DataFrame(columns=['Open','High','Low','Close','Volume','Complete'])
        self.available_intervals = ['1m','3m','5m','15m','30m','1h','2h','4h','6h','8h','12h','1d','3d','1w','1M']
        
    def start_trading(self):
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        if self.bar_length in self.available_intervals:
            self.twm.start_kline_socket(callback = self.stream_candles,symbol =self.symbol,interval=self.bar_length)
            
    def stream_candles(self,msg):
        
        event_time = pd.to_datetime(msg['E'],unit='ms')
        start_time = pd.to_datetime(msg['k']['t'],unit='ms')
        first = float(msg['k']['o'])
        high = float(msg['k']['h'])
        low = float(msg['k']['l'])
        close = float(msg['k']['c'])
        volume = float(msg['k']['v'])
        complete = float(msg['k']['x'])
        
        print("Time: {} | Price: {}".format(event_time, close))
        
        self.data.loc[start_time] = [first, high, low, close, volume, complete]

In [37]:
trader = LongOnlyTrader(symbol ='BTCUSDT',bar_length='1m')

In [33]:
trader.symbol

'BTCUSDT'

In [34]:
trader.data

['Open', 'High', 'Low', 'Close', 'Volume', 'Complete']

In [38]:
trader.start_trading()

Time: 2022-02-25 17:55:23.196000 | Price: 38904.62
Time: 2022-02-25 17:55:26.007000 | Price: 38904.61
Time: 2022-02-25 17:55:28.210000 | Price: 38891.6
Time: 2022-02-25 17:55:30.210000 | Price: 38889.56
Time: 2022-02-25 17:55:32.315000 | Price: 38888.98


In [39]:
trader.twm.stop()

Time: 2022-02-25 17:57:06.340000 | Price: 38932.27


In [40]:
trader.data

,Open,High,Low,Close,Volume,Complete
2022-02-25 17:55:00,38908.48,38928.06,38872.70,38882.77,64.68505,1.0
2022-02-25 17:56:00,38882.77,38944.62,38882.76,38920.49,98.57170,1.0
2022-02-25 17:57:00,38920.50,38945.98,38920.49,38932.27,6.43586,0.0


In [ ]:
##Historcal data and real time data
#merge historal data with live data  

In [41]:
from datetime import datetime,timedelta

In [42]:
now = datetime.utcnow()

In [43]:
past = now - timedelta(days=2)

In [46]:
def get_most_recent(symbol,interval,days):
    now = datetime.utcnow()
    past =str(now - timedelta(days=days))
    
    bars = client.get_historical_klines(symbol=symbol,interval=interval,start_str=past,end_str=None,limit =1000)
    df = pd.DataFrame(bars)
    df['Date'] = pd.to_datetime(df.iloc[:,0],unit='ms')
    df.columns =['Open Time','Open','High','Low','Close','Volume','Close Time','Quote Asset Volume','Number of Trades','Taker Buy Base Asset Volume','Taker Buy Quote Asset Volume','Ignore','Date']
    df = df[['Date','Open','High','Low','Close','Volume']]
    df.set_index('Date',inplace=True)
    for i in df.columns:
        df[i] = pd.to_numeric(df[i],errors='coerce')
    df['Complete'] = [True for row in range(len(df)-1)]+ [False]
    return df
    

In [47]:
df = get_most_recent('BTCUSDT','1m',2)

In [48]:
df

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2022-02-23 18:06:00,38026.05,38040.05,37943.66,37967.74,4.202172,True
2022-02-23 18:07:00,37967.74,38030.64,37967.74,37990.84,3.838130,True
2022-02-23 18:08:00,37990.84,37996.11,37943.78,37957.00,2.997902,True
2022-02-23 18:09:00,37955.51,37977.73,37954.05,37977.73,3.762367,True
2022-02-23 18:10:00,37976.53,38002.08,37971.52,37981.19,2.932382,True
...,...,...,...,...,...,...
2022-02-25 18:01:00,39011.64,39036.50,38990.77,38990.77,3.937058,True
2022-02-25 18:02:00,38990.77,38990.77,38893.24,38893.25,3.216573,True
2022-02-25 18:03:00,38897.81,38928.35,38785.64,38928.35,5.459557,True


In [50]:
class LongOnlyTrader():
    
    def __init__(self,symbol,bar_length):
        self.symbol = symbol
        self.bar_length = bar_length
        self.data = pd.DataFrame(columns=['Open','High','Low','Close','Volume','Complete'])
        self.available_intervals = ['1m','3m','5m','15m','30m','1h','2h','4h','6h','8h','12h','1d','3d','1w','1M']
        
    def start_trading(self,historical_days):
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol=self.symbol,interval = self.bar_length,days=historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,symbol =self.symbol,interval=self.bar_length)
            
    def get_most_recent(self,symbol,interval,days):
        now = datetime.utcnow()
        past =str(now - timedelta(days=days))
        
        bars = client.get_historical_klines(symbol=symbol,interval=interval,start_str=past,end_str=None,limit =1000)
        df = pd.DataFrame(bars)
        df['Date'] = pd.to_datetime(df.iloc[:,0],unit='ms')
        df.columns =['Open Time','Open','High','Low','Close','Volume','Close Time','Quote Asset Volume','Number of Trades','Taker Buy Base Asset Volume','Taker Buy Quote Asset Volume','Ignore','Date']
        df = df[['Date','Open','High','Low','Close','Volume']]
        df.set_index('Date',inplace=True)
        for i in df.columns:
            df[i] = pd.to_numeric(df[i],errors='coerce')
        df['Complete'] = [True for row in range(len(df)-1)]+ [False]
        self.data = df        
    
    def stream_candles(self,msg):
        
        event_time = pd.to_datetime(msg['E'],unit='ms')
        start_time = pd.to_datetime(msg['k']['t'],unit='ms')
        first = float(msg['k']['o'])
        high = float(msg['k']['h'])
        low = float(msg['k']['l'])
        close = float(msg['k']['c'])
        volume = float(msg['k']['v'])
        complete = float(msg['k']['x'])
        
        print("Time: {} | Price: {}".format(event_time, close))
        
        self.data.loc[start_time] = [first, high, low, close, volume, complete]

In [51]:
trader = LongOnlyTrader(symbol='BTCUSDT',bar_length='1m')

In [52]:
trader.start_trading(historical_days=2)

Time: 2022-02-25 18:10:54.925000 | Price: 38642.83
Time: 2022-02-25 18:10:57.022000 | Price: 38641.77
Time: 2022-02-25 18:10:59.151000 | Price: 38641.77
Time: 2022-02-25 18:11:00.002000 | Price: 38645.47
Time: 2022-02-25 18:11:02.003000 | Price: 38631.92
Time: 2022-02-25 18:11:04.074000 | Price: 38634.24
Time: 2022-02-25 18:11:06.095000 | Price: 38621.1


In [53]:
trader.twm.stop()

Time: 2022-02-25 18:12:25.423000 | Price: 38669.78


In [54]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2022-02-23 18:11:00,37980.52,37980.52,37915.57,37915.57,3.314605,True
2022-02-23 18:12:00,37915.58,37923.46,37887.10,37901.25,3.963603,True
2022-02-23 18:13:00,37901.25,37907.05,37846.11,37886.19,4.362019,True
2022-02-23 18:14:00,37887.11,37887.11,37846.02,37846.19,2.933487,True
2022-02-23 18:15:00,37846.15,37887.06,37846.01,37874.47,3.634838,True
...,...,...,...,...,...,...
2022-02-25 18:08:00,38781.60,38781.60,38533.29,38606.38,5.085175,True
2022-02-25 18:09:00,38606.38,38620.87,38562.02,38598.75,4.228756,True
2022-02-25 18:10:00,38603.33,38660.00,38555.55,38645.47,73.145480,1.0


In [64]:
import numpy as np

# Adding Long Only Strategy

In [80]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
        
    def define_strategy(self): # "strategy-specific"
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
        

In [81]:
symbol = 'BTCUSDT'
bar_length ='1m'
return_thresh = 0
volume_thresh =[-3,3]

In [82]:
trader = LongOnlyTrader(symbol=symbol,bar_length=bar_length,return_thresh=return_thresh,volume_thresh=volume_thresh)

In [85]:
trader.start_trading(historical_days=1/24)

Task was destroyed but it is pending!
task: <Task pending name='Task-660' coro=<ReconnectingWebsocket._read_loop() done, defined at C:\Users\jdlugosz\AppData\Local\Programs\Python\Python38\lib\site-packages\binance\streams.py:122> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x000001A1D19BD100>()]>>


..........................

In [86]:
trader.twm.stop()


.

In [87]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2022-02-25 18:07:00,38845.19,38890.15,38764.87,38777.12,4.140148,True
2022-02-25 18:08:00,38781.60,38781.60,38533.29,38606.38,5.085175,True
2022-02-25 18:09:00,38606.38,38620.87,38562.02,38598.75,4.228756,True
2022-02-25 18:10:00,38596.54,38656.36,38560.22,38645.47,4.672443,True
2022-02-25 18:11:00,38641.77,38645.48,38595.61,38621.30,4.019696,True
...,...,...,...,...,...,...
2022-02-25 19:03:00,38709.37,38715.06,38693.73,38709.66,2.579860,True
2022-02-25 19:04:00,38709.57,38719.97,38674.03,38674.05,3.217785,True
2022-02-25 19:05:00,38672.65,38686.71,38662.47,38684.70,2.790514,True


In [88]:
trader.prepared_data

,Close,Volume,returns,vol_ch,position
Date,,,,,
2022-02-25 18:07:00,38777.12,4.140148,NaN,NaN,1
2022-02-25 18:08:00,38606.38,5.085175,-0.004413,0.205598,1
2022-02-25 18:09:00,38598.75,4.228756,-0.000198,-0.184422,1
2022-02-25 18:10:00,38645.47,4.672443,0.001210,0.099774,0
2022-02-25 18:11:00,38621.30,4.019696,-0.000626,-0.150476,1
2022-02-25 18:12:00,38673.73,3.621892,0.001357,-0.104210,0
2022-02-25 18:13:00,38651.00,3.472774,-0.000588,-0.042043,1
2022-02-25 18:14:00,38631.28,3.318577,-0.000510,-0.045418,1
2022-02-25 18:15:00,38637.73,2.745807,0.000167,-0.189461,0


In [89]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units # NEW
        self.position = position # NEW
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades() # NEW!!!
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): # NEW!
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                print("GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                print("GOING NEUTRAL")
            self.position = 0

Task was destroyed but it is pending!
task: <Task pending name='Task-644' coro=<ReconnectingWebsocket._read_loop() running at C:\Users\jdlugosz\AppData\Local\Programs\Python\Python38\lib\site-packages\binance\streams.py:130> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x000001A1D15019D0>()]>>


In [93]:
symbol = 'BTCUSDT'
bar_length ='1m'
return_thresh = 0
volume_thresh =[-3,3]
units = 0.01 
position = 0

In [94]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1645761041984,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.90000000', 'locked': '0.20000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '3079.82159717', 'locked': '3000.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [95]:
trader = LongOnlyTrader(symbol=symbol,bar_length=bar_length,return_thresh=return_thresh,volume_thresh=volume_thresh,units=units,position=position)

In [98]:
trader.start_trading(historical_days=1/24)

.............................................GOING LONG
.................

In [99]:
trader.twm.stop()

.

In [100]:
trader.prepared_data

,Close,Volume,returns,vol_ch,position
Date,,,,,
2022-02-25 18:18:00,38768.48,4.198415,NaN,NaN,1
2022-02-25 18:19:00,38780.39,3.406835,0.000307,-0.208923,0
2022-02-25 18:20:00,38753.42,2.864252,-0.000696,-0.173476,1
2022-02-25 18:21:00,38794.59,4.048011,0.001062,0.345918,0
2022-02-25 18:22:00,38776.21,4.823768,-0.000474,0.175330,1
...,...,...,...,...,...
2022-02-25 19:14:00,38668.23,3.038374,-0.000673,-0.008770,1
2022-02-25 19:15:00,38713.01,3.695519,0.001157,0.195799,0
2022-02-25 19:16:00,38752.35,2.657162,0.001016,-0.329862,0


In [101]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1645816740182,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.91000000', 'locked': '0.20000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '2692.54319717', 'locked': '3000.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

# Trade Monitoring

In [102]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.position = position
        self.trades = 0 # NEW
        self.trade_values = [] # NEW
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")  # NEW
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            self.position = 0
    
    def report_trade(self, order, going):  # NEW
        
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units / base_units, 5)
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, cum_profits))
        print(100 * "-" + "\n")

Task was destroyed but it is pending!
task: <Task pending name='Task-763' coro=<ReconnectingWebsocket._read_loop() running at C:\Users\jdlugosz\AppData\Local\Programs\Python\Python38\lib\site-packages\binance\streams.py:130> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x000001A1D19320A0>()]>>


In [104]:
symbol = 'BTCUSDT'
bar_length ='1m'
return_thresh = 0
volume_thresh =[-3,3]
units = 0.01 
position = 0

In [105]:
trader = LongOnlyTrader(symbol=symbol,bar_length=bar_length,return_thresh=return_thresh,volume_thresh=volume_thresh,units=units,position=position)

In [103]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1645816740182,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.91000000', 'locked': '0.20000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '2692.54319717', 'locked': '3000.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [106]:
trader.start_trading(historical_days=1/24)

.................

----------------------------------------------------------------------------------------------------
2022-02-25 19:24:00.633000 | GOING LONG
2022-02-25 19:24:00.633000 | Base_Units = 0.01 | Quote_Units = 389.0489 | Price = 38904.89 
2022-02-25 19:24:00.633000 | Profit = 0 | CumProfits = 0.0 
----------------------------------------------------------------------------------------------------

.............................

----------------------------------------------------------------------------------------------------
2022-02-25 19:25:00.179000 | GOING NEUTRAL
2022-02-25 19:25:00.179000 | Base_Units = 0.01 | Quote_Units = 389.4568 | Price = 38945.68 
2022-02-25 19:25:00.179000 | Profit = 0.408 | CumProfits = 0.408 
----------------------------------------------------------------------------------------------------

............................

----------------------------------------------------------------------------------------------------
2022-02-25 19:26:00.

In [107]:
trader.twm.stop()

.

In [110]:
np.sum(trader.trade_values)

-388.5234

In [109]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1645817340426,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.92000000', 'locked': '0.20000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '2304.01979717', 'locked': '3000.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

## Stopping Trades

In [121]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.position = position
        self.trades = 0 
        self.trade_values = [] 
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # stop trading session
        if event_time >= datetime(2022, 2, 25, 20, 55):
            self.twm.stop()
            if self.position != 0:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL AND STOP") 
                self.position = 0
            else: 
                print("STOP")
        
        #more stop examples:
        # if self.trades >= xyz
        # if self.cum_profits <> xyz
        
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0
    
    def report_trade(self, order, going):
        
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units / base_units, 5)
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            self.cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            self.cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")

In [122]:
symbol = 'BTCUSDT'
bar_length ='1m'
return_thresh = 0
volume_thresh =[-3,3]
units = 0.01 
position = 0

In [123]:
trader = LongOnlyTrader(symbol=symbol,bar_length=bar_length,return_thresh=return_thresh,volume_thresh=volume_thresh,units=units,position=position)

In [124]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1645817639015,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.92000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10304.01979717', 'locked': '3000.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [125]:
trader.start_trading(historical_days=1/24)

...........

----------------------------------------------------------------------------------------------------
2022-02-25 19:37:01.705000 | GOING LONG
2022-02-25 19:37:01.705000 | Base_Units = 0.01 | Quote_Units = 387.2512 | Price = 38725.12 
2022-02-25 19:37:01.705000 | Profit = 0 | CumProfits = 0.0 
----------------------------------------------------------------------------------------------------

.......................................................

----------------------------------------------------------------------------------------------------
2022-02-25 19:39:01.356000 | GOING NEUTRAL
2022-02-25 19:39:01.356000 | Base_Units = 0.01 | Quote_Units = 387.2412 | Price = 38724.12 
2022-02-25 19:39:01.356000 | Profit = -0.01 | CumProfits = -0.01 
----------------------------------------------------------------------------------------------------

..................................................................................

-----------------------------------------------

In [126]:
trader.twm.stop()

.

In [127]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'updateTime': 1645818240201,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '1.02000000', 'locked': '0.00000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '10303.36439717', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [128]:
trader.cum_profits

-0.655